In [1]:
import sys
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.regularizers import l2 
from keras.models import Sequential, Model,load_model

## data loading


In [2]:
data = open('shahnameh.txt', 'r', encoding="utf8").read().lower()

In [3]:
print(type(data))
print(len(data))

<class 'str'>
2653849


In [4]:
unique_chars = set(data)
print('unique chars are:\n', unique_chars)

unique chars are:
 {'أ', ' ', 'ی', 'س', 'ض', 'ح', 'ه', 'م', 'خ', 'ف', '(', 'ٔ', 'غ', 'ت', '«', '؟', 'ل', 'ط', 'چ', 'ک', 'د', 'گ', 'پ', '|', 'ز', '»', 'ذ', 'ن', 'ب', 'آ', 'ئ', 'ع', ')', 'ا', 'ق', '\n', 'ء', 'ث', '\u200c', 'ؤ', 'و', 'ص', '،', 'ظ', 'ج', 'ر', 'ش', 'ژ'}


In [5]:
vocab = sorted(unique_chars)
vocab_size = len(vocab)
char2idx = {u:i for i, u in enumerate(vocab)}   # saving characters into a dictionary
idx2char = np.array(vocab)                      # saving characters into a numpy array

In [6]:
print(type(char2idx))
print(char2idx)

<class 'dict'>
{'\n': 0, ' ': 1, '(': 2, ')': 3, '|': 4, '«': 5, '»': 6, '،': 7, '؟': 8, 'ء': 9, 'آ': 10, 'أ': 11, 'ؤ': 12, 'ئ': 13, 'ا': 14, 'ب': 15, 'ت': 16, 'ث': 17, 'ج': 18, 'ح': 19, 'خ': 20, 'د': 21, 'ذ': 22, 'ر': 23, 'ز': 24, 'س': 25, 'ش': 26, 'ص': 27, 'ض': 28, 'ط': 29, 'ظ': 30, 'ع': 31, 'غ': 32, 'ف': 33, 'ق': 34, 'ل': 35, 'م': 36, 'ن': 37, 'ه': 38, 'و': 39, 'ٔ': 40, 'پ': 41, 'چ': 42, 'ژ': 43, 'ک': 44, 'گ': 45, 'ی': 46, '\u200c': 47}


In [7]:
print(type(idx2char))
print(idx2char)

<class 'numpy.ndarray'>
['\n' ' ' '(' ')' '|' '«' '»' '،' '؟' 'ء' 'آ' 'أ' 'ؤ' 'ئ' 'ا' 'ب' 'ت' 'ث'
 'ج' 'ح' 'خ' 'د' 'ذ' 'ر' 'ز' 'س' 'ش' 'ص' 'ض' 'ط' 'ظ' 'ع' 'غ' 'ف' 'ق' 'ل'
 'م' 'ن' 'ه' 'و' 'ٔ' 'پ' 'چ' 'ژ' 'ک' 'گ' 'ی' '\u200c']


In [8]:
text_as_int = np.array([char2idx[c] for c in data])    # integer-encoded dataset 

In [9]:
print(len(data))
print(len(text_as_int))

2653849
2653849


In [10]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)      # char_dataset is an object holding dataset!

In [11]:
print(len(text_as_int))
print(len(char_dataset))

2653849
2653849


In [12]:
sequences = char_dataset.batch(10001, drop_remainder=True)             # seq_length = 1000

In [13]:
len(sequences)     # len(char_dataset)= 2653849  , seq_length = 1000   --->   int(2653849/1001) = 2651

265

In [14]:
def split_input_target(sequence):
    input_text = sequence[:-1]         # to extract from index  0 to 999
    target_text = sequence[1:]         # to extract from index  1 to 1000
    return input_text, target_text

In [15]:
dataset = sequences.map(split_input_target)   # X, Y  are in the "dataset" object!  

In [16]:
len(dataset)

265

In [17]:
for element in dataset:       # each element in the "dataset" is a tuple: (input_vector, output_vector) 
    
    print(len(element))
    
    print( element[0].shape)  # input  ( 1000 x 1 ) vector 
    print( element[1].shape)  # output ( 1000 x 1 ) vector 
    break

2
(10000,)
(10000,)


In [18]:
dataset = dataset.shuffle(10000).batch(64, drop_remainder=True)       # BATCH_SIZE=64

In [19]:
len(dataset)    # len(sequences) = 2651  ,   BATCH_SIZE=64  ---->   int(2651/64) = 41

4

In [20]:
for element in dataset:       # element(input_batch_matrix , output_batch_matrix)
    
    print(len(element))

    print( element[0].shape)  # input   batch matrix : ( 64 x 1000 )
    print( element[1].shape)  # output  batch matrix : ( 64 x 1000 )

    print( element[0][5].shape)   # input  vector in the 5'th batch
    print( element[1][38].shape)  # output vector in the 38'th batch
    break

2
(64, 10000)
(64, 10000)
(10000,)
(10000,)


##### Up to this point, all the data has been converted into the required format for training the network and stored in the variable `dataset`. Both input and output data will be provided to the RNN in the form of 1000-dimensional vectors in batches of 64. Each element inside the input vectors corresponds to a numerical value representing a character. Since each character must be converted into a vector to be fed into the network, an embedding layer is added at the beginning of the RNN. We have set the `embedding_dim` parameter for the embedding layer to 256. This way, this layer will convert the numerical value of each input character into a vector in a 256-dimensional space. Then this 256-dimensional vector is input into 2 LSTM layers, followed by a dense layer that converts the output of the RNN into a 48-dimensional vector, equal to the total number of characters in the dataset. Each of the LSTM layers has 1024 neurons.


In [21]:
# vocab_length = 48
# BATCH_SIZE = 64
# embedding_dim = 256
# seq_length = 1000

In [22]:
model = Sequential([
    
  layers.Embedding( 48, 256, batch_input_shape=[64, None]),  # "None" has placed because input sequence can have differnet lengths!

  layers.LSTM(1024,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)),
  layers.LSTM(1024,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)),

  layers.Dense(48)             

  ])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           12288     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 48)            49200     
                                                                 
Total params: 13,701,168
Trainable params: 13,701,168
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [39]:
history = model.fit(dataset, epochs=50)

Epoch 1/50
41/41 [==============================] - 75s 2s/step - loss: 13.4405 - accuracy: 0.1583
Epoch 2/50
41/41 [==============================] - 67s 2s/step - loss: 3.4817 - accuracy: 0.2226
Epoch 3/50
41/41 [==============================] - 68s 2s/step - loss: 2.7445 - accuracy: 0.2757
Epoch 4/50
41/41 [==============================] - 67s 2s/step - loss: 2.4732 - accuracy: 0.3047
Epoch 5/50
41/41 [==============================] - 68s 2s/step - loss: 2.3686 - accuracy: 0.3306
Epoch 6/50
41/41 [==============================] - 68s 2s/step - loss: 2.2943 - accuracy: 0.3483
Epoch 7/50
41/41 [==============================] - 68s 2s/step - loss: 2.2130 - accuracy: 0.3668
Epoch 8/50
41/41 [==============================] - 68s 2s/step - loss: 2.1434 - accuracy: 0.3818
Epoch 9/50
41/41 [==============================] - 67s 2s/step - loss: 2.0783 - accuracy: 0.3950
Epoch 10/50
41/41 [==============================] - 68s 2s/step - loss: 2.0213 - accuracy: 0.4093
Epoch 11/50
41/41 

In [23]:
model = load_model("650epochs.hdf5")

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12288     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 48)            49200     
Total params: 13,701,168
Trainable params: 13,701,168
Non-trainable params: 0
_________________________________________________________________


### To generate new data, we create a network similar to the one built for training. The difference is that we set the input dimensions to the size of a sequence instead of a batch size, because the input text to the network for generating verses will be in the form of a sequence.


In [25]:
model2 = Sequential([
    
  layers.Embedding(48, 256, batch_input_shape=[1, None]),

  layers.LSTM(1024,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)),
  layers.LSTM(1024,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)),

  layers.Dense(48)             

  ])

In [26]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 48)             49200     
Total params: 13,701,168
Trainable params: 13,701,168
Non-trainable params: 0
_________________________________________________________________


### Transferring trained parameters to the second model

In [27]:
model2.set_weights(model.get_weights())

In [28]:
def generate_text(model, start_string): 
    
    # lets say the length of start_string is equal to "L"
 
    # num_gen: number of characters to generate
    num_gen = 1000

    #---------------------------------------------------------------------------------------#
    # input_eval here is a "list" of star_string character encoded numbers  
    input_eval = [char2idx[s] for s in start_string] 

    # converting the input_eval list to a (1,L) tensorflow tensor as an input to the network
    input_eval = tf.expand_dims(input_eval, 0) 
    #---------------------------------------------------------------------------------------#
    # resetting LSTM cell memories   
    model.reset_states()  
    # ---------------------------------------------------------------------------------------#
    # an empty list for saving ganerated texts
    generated_text = []
  #---------------------------------------------------------------------------------------#

    for i in range(num_gen):
        
        # model output is a tensorflow tensor shaped ([1, L, 48])  
        predictions = model(input_eval)   
        
        # turning the shape of the output tensor into ([L, 48])   
        predictions = tf.squeeze(predictions, 0)
        
        #  drawing an independent sample from each row of output matrix(prediction)
        #  and extracting the choosen element from the last row ( the last output of network )
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # adding generated new character to the text
        generated_text.append(idx2char[predicted_id])

        # creating new input_eval with predicted_id in a tensorflow tensor shape as a new input to the network
        input_eval = tf.expand_dims([predicted_id], 0)
    
  #---------------------------------------------------------------------------------------#
    return (start_string + ''.join(generated_text))

In [31]:
rnn_poems = generate_text(model2, start_string="بدیشان چنین گفت پس شهریار")

In [32]:
print(rnn_poems)

بدیشان چنین گفت پس شهریار
|که با ایمنی شهنشاه یمن
|که بیزارم از دل خویشتن
|کنون باز پاسبان گرفتند
|زمین چون قو بشنوده فسون
|چودلش کرا به‌هم شاه انجمن
|همی موی پاک یاد از مهر اوی
|من این چادر قیصر اندکی
|کشان و دل پر ز گور اندکی
|نشد سست و او را به آواز نرم
|چنین گفت با خاک و آب کردش کس
|من این پادا میان را ببست
|پذیرم من نیک نام او اردشیر
|وگرنه بفرمود با سوار
|مبادا که باشیم شاه و ترکان و چین
|گسسته نبود ای سر ما زمان
|که یابد نشانی زمان همی
|از انجمن سخن تو پاکیزه‌رای
|چو افگند سینه و کدخدای
|به پهنای پادشاهی و راست
|سه چرخ بلندگان ما
|ز هندوستان بما نام تو
|بفرمود تا به اژدها
|نیامد به فرجام و تاج
|خنک بودش آبگنگ ای سوار
|چنین گوی کاین زهر آورد
|بدان خواسته یک بار درخورد
|سر آنکه ه سوگند خوار
|همی پروراندهٔ روز ناسودمند
|چنین پاسخ آوردش اسفندیار
|که ای از توآمد همی با فسوس
|سپاهم فرستد او را ببست
|همان از پسشگفت یبدست
|جهانی ببندوی او برید
|سر چاه بن
|پذیره شو و پهلوان سپاه
|نخواهد ز منش و کاموس بود
|جوان بود مر پاک دخدای
|که دانا پیر سرافراز شاه
|فراوان بلند ازین جاماسپ را
|شنیدم ک

### Evaluation 

### یکی از ابزارهای ارزیابی مدل های تولید کننده متن sentence_bleu می باشد
### این ابزار دو متن را به عنوان ورودی گرفته و عددی بین 0 و 1 بر می گرداند. عدد تولید شده نشانگر میزان شباهت موجود میان دو متن می باشد

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [ ]:
rnn_poems = generate_text(model2, start_string="به نام خداوند جان و خرد")

In [ ]:
print(rnn_poems)

به نام خداوند جان و خرد
|نه از من چه آزردهٔ ماه اورند
|بدو گفت گیو این سخن شهریار
|پر اندیشه با ناله منست
|برفت و بدین باز گشتی جهان
|چنانش بدو ای دل پاک
|ز خون ریختن با تو تافت
|نه کشور چنین یکی ناسودمند
|بگوی و خاک را بنهید روی
|دوتا جهان از بد او یارتوان
|بدین زور و این کاهلی
|تو شو تیز خویشتن دیده‌ای
|به گیتی جز از پهین داستان
|نبودند کین از شیر آژیر بایدمی
|که شد مغز ایشان فرخ نهیم
|بپیش بزنار اندر از شرم
|وزان پس که پایگها بشور بخواه
|شگفتی بجوش آمد از کوه و دود
|رخ مادر ار بدی تاب آورد که کشت
|سزا و ز کیخسرو آغاز تو
|چنین داد پاسخ که ای پهلوان
|نباشد بدین ره تا باید گریست
|چو آن گشت از ایرج آن تست
|ز مهر ومایگاه آراستستی
|نخواهد ز بخشش سر تاج زر
|وگر هیچ دارید تا چه گونه نهنگ
|ندانی همی تا بگرم نیکی بخواه
|چو با سپه دد پادشاهی سپاه
|ندیدم چون کین بود ایزدی
|که بخشنده اویست و دستبرد
|بوردم به آب داده اندر موبد
|بدو گفت گستهم کای شهریار
|شوم گفت مانامه بازجست
|چنین است پیروزگر
|پریچید گردن میان من
|ز خویشان می پاسبان سپهر
|بدو گفت گازر که از پهلوان
|که با ما چون روانم پیشرو
|تو دل

In [ ]:
sentence_bleu(data[:1000] , rnn_poems, smoothing_function=SmoothingFunction().method2)

0.0022502591251490176